# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
filepath = song_files[0]

In [6]:
df = pd.read_json(filepath, lines = True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
df.values[0].tolist()

[1,
 'AR7G5I41187FB4CE6C',
 nan,
 nan,
 'London, England',
 'Adam Ant',
 'SONHOTT12A8C13493C',
 'Something Girls',
 233.40363,
 1982]

In [8]:
values = {}
for column in df.columns:
    values[column] = df[column].values[0]
print(values)

{'num_songs': 1, 'artist_id': 'AR7G5I41187FB4CE6C', 'artist_latitude': nan, 'artist_longitude': nan, 'artist_location': 'London, England', 'artist_name': 'Adam Ant', 'song_id': 'SONHOTT12A8C13493C', 'title': 'Something Girls', 'duration': 233.40363, 'year': 1982}


In [9]:
for key,v in values.items():
    print(v, type(v))

1 <class 'numpy.int64'>
AR7G5I41187FB4CE6C <class 'str'>
nan <class 'numpy.float64'>
nan <class 'numpy.float64'>
London, England <class 'str'>
Adam Ant <class 'str'>
SONHOTT12A8C13493C <class 'str'>
Something Girls <class 'str'>
233.40363 <class 'numpy.float64'>
1982 <class 'numpy.int64'>


In [10]:
song_data = [values['song_id'], values['title'], values['artist_id'], int(values['year']), float(values['duration'])]
song_data

['SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [11]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [12]:
artist_data = [values['artist_id'], values['artist_name'], values['artist_location'], float(values['artist_longitude']), float(values['artist_latitude'])]
artist_data

['AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [13]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [14]:
log_files = get_files('data/log_data')

In [15]:
filepath = log_files[0]
filepath

'/Users/touchbar/Desktop/Data Engineer Udacity/Project01-Data-Modeling-Postgres/data/log_data/2018/11/2018-11-11-events.json'

In [16]:
df = pd.read_json(filepath, lines = True)
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 18 columns):
artist           73 non-null object
auth             95 non-null object
firstName        92 non-null object
gender           92 non-null object
itemInSession    95 non-null int64
lastName         92 non-null object
length           73 non-null float64
level            95 non-null object
location         92 non-null object
method           95 non-null object
page             95 non-null object
registration     92 non-null float64
sessionId        95 non-null int64
song             73 non-null object
status           95 non-null int64
ts               95 non-null int64
userAgent        92 non-null object
userId           95 non-null object
dtypes: float64(2), int64(4), object(12)
memory usage: 13.5+ KB


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [17]:
df = df[df['page']=='NextSong']
df.shape

(73, 18)

In [18]:
t = pd.to_datetime(df['ts'], unit='ms') 
t.head()

0   2018-11-11 02:33:56.796
1   2018-11-11 02:36:10.796
2   2018-11-11 02:40:34.796
4   2018-11-11 04:36:13.796
5   2018-11-11 04:36:46.796
Name: ts, dtype: datetime64[ns]

In [19]:
time_data = []
for line in t:
    time_data.append([line, 
                      line.hour, 
                      line.day, 
                      line.week, 
                      line.month, 
                      line.year, 
                      line.day_name()])
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [20]:
time_df = pd.DataFrame.from_records(time_data, columns=column_labels)
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-11 02:33:56.796,2,11,45,11,2018,Sunday
1,2018-11-11 02:36:10.796,2,11,45,11,2018,Sunday
2,2018-11-11 02:40:34.796,2,11,45,11,2018,Sunday
3,2018-11-11 04:36:13.796,4,11,45,11,2018,Sunday
4,2018-11-11 04:36:46.796,4,11,45,11,2018,Sunday


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [21]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, row)
    conn.commit()
    #print(i,list(row))

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [22]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
1,69,Anabelle,Simpson,F,free
2,69,Anabelle,Simpson,F,free
4,32,Lily,Burns,F,free
5,75,Joseph,Gutierrez,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [23]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [25]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ( pd.to_datetime(row.ts, unit='ms'), 
                     int(row.userId), 
                     row.level, 
                     songid, 
                     artistid, 
                     row.sessionId, 
                     row.location, 
                     row.userAgent)
    
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [25]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

# Testing the ON CONFLICT on the user table

In [23]:
user_df.values[0]

array(['69', 'Anabelle', 'Simpson', 'F', 'free'], dtype=object)

In [24]:
columns = ['userId', 'firstName', 'lastName', 'gender', 'level']
foo = pd.DataFrame([['69', 'Anabelle', 'Simpson', 'F', 'free'],['69', 'Anabelle', 'Simpson', 'F', 'paid']], columns = columns)
foo.head()

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
1,69,Anabelle,Simpson,F,paid


In [25]:
for i, row in foo.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()